In [30]:
import requests
import json
import pandas as pd

from collections import Counter

In [23]:
SCORING_URL = 'http://a6b79cd8-c6ce-4fa8-926c-2f22aeb185fb.eastus.azurecontainer.io/score'

In [24]:
amazon_valid = pd.read_csv('../data/amazon_valid.csv')

In [25]:
amazon_valid.head()

,id,text,label
0,0,We started giving this to my 5 year old Labrad...,dogs
1,1,This product is easy to set up and use. I have...,dogs
2,2,my dog has nerves and wants to itch and chew.....,dogs
3,3,the breeded gave us a can of Nupro when we too...,dogs
4,4,"Very study, well made poop bag. Easy to open a...",dogs


In [26]:
len(set(amazon_valid['label']))

6

In [27]:
def get_scored_label(df):
    data = []
    for _, row in df.iterrows():
        data.append({'text':row['text'], 'id':row['id']})
    
    # Two sets of data to score, so we get two results back
    data = {"data": data}
    # Convert to JSON string
    input_data = json.dumps(data)

    # Set the content type
    headers = {'Content-Type': 'application/json'}

    # Make the request and display the response
    resp = requests.post(SCORING_URL, input_data, headers=headers)
    
    output = json.loads(resp.json())['result']
    df['PredictedLabel']=output
    return df

In [32]:
processed_df = get_scored_label(amazon_valid.head(150))

c:\users\ajakupov\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
processed_df['isCorrect']=processed_df['label']==processed_df['PredictedLabel']

c:\users\ajakupov\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
Counter(processed_df['isCorrect'])

Counter({True: 119, False: 31})